In [1]:
# PySpark sessions for CPU and GPU
from pyspark.sql import SparkSession

# for CPU
"""
spark = SparkSession.builder \
	.master("local").appName("hdfs_test")\
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9009") \
    .getOrCreate()
"""

# for GPU
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("hdfs_test") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.task.cpus", "4") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.rapids.memory.gpu.pool.size", "2G") \
    .config("spark.driver.memory", "12g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/06 07:11:14 WARN Utils: Your hostname, MyPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/06 07:11:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/06 07:11:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
silver_layer_path = "file:///home/tim/al"
songs_embedded = spark.read.parquet(f"{silver_layer_path}/song_info_embedded").orderBy('faiss_id')
songs_embedded.show(3)

25/05/06 07:11:16 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/05/06 07:11:16 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/05/06 07:11:16 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


+--------------------+-------------------+-------+------------------+--------------------+-----+--------+------+------------+------------+-----------+--------+------------+----------------+--------------------+--------+
|              artist|               song|emotion|          variance|               Genre|Tempo|Loudness|Energy|Danceability|Positiveness|Speechiness|Liveness|Acousticness|Instrumentalness|     embedded_vector|faiss_id|
+--------------------+-------------------+-------+------------------+--------------------+-----+--------+------+------------+------------+-----------+--------+------------+----------------+--------------------+--------+
|  (Alla Pugacheva...|Позови меня P.I.M.P|  anger|0.8335142456515902|Unknown,Unknown,U...|  175|   -7.15|    91|          42|          32|          8|      38|           0|              84|[-0.023154184, 0....|       0|
|         (Ani Lorak)|         Shady Lady|    joy|0.8335142456515902|Unknown,Unknown,U...|  128|   -13.0|    70|        

In [3]:
# CONVERT TO LIST OF VECTORS TO STORE IN FAISS

EMD_COL_NAME = "embedded_vector"

embedding_rows = songs_embedded.select(EMD_COL_NAME).collect()
embedding_vectors = [row[EMD_COL_NAME] for row in embedding_rows]

In [10]:
# STORE IN FAISS
import faiss
import numpy as np

def normalize(vectors):
    return vectors / np.linalg.norm(vectors, keepdims=True)

embedding_matrix = np.array(embedding_vectors).astype("float32")

normalized_vectors = normalize(embedding_matrix)

index = faiss.IndexFlatIP(normalized_vectors.shape[1])
index.add(normalized_vectors)

print(f"FAISS index created with {index.ntotal} vectors.")

FAISS index created with 236600 vectors.


In [ ]:
faiss.write_index(index, "faiss_song_index.index")
print("FAISS index saved to 'faiss_song_index.index'")

FAISS index saved to 'faiss_song_index.index'


In [ ]:
books_df = spark.read.parquet(f"{silver_layer_path}/book_total_info")
books_df.show(10)

+--------+--------------------+----------+--------------------+----------+
| book_id|               title|      isbn|         description|     genre|
+--------+--------------------+----------+--------------------+----------+
|10000063|The Two Deaths of...|052595211X|Marcus Sakey retu...|   fiction|
|10000063|The Two Deaths of...|052595211X|Marcus Sakey retu...|   mystery|
|10000063|The Two Deaths of...|052595211X|Marcus Sakey retu...|  thriller|
|10000063|The Two Deaths of...|052595211X|Marcus Sakey retu...|     crime|
|10000294|     The Hoopicopter|0316903922|                    |  children|
|10000597| Crooked Little Vein|1400175615|Michael McGill is...|    comics|
|10000597| Crooked Little Vein|1400175615|Michael McGill is...|   graphic|
|10000597| Crooked Little Vein|1400175615|Michael McGill is...|   fantasy|
|10000597| Crooked Little Vein|1400175615|Michael McGill is...|paranormal|
|10000597| Crooked Little Vein|1400175615|Michael McGill is...|   fiction|
+--------+---------------

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

from pyspark.sql.functions import pandas_udf
import pandas as pd

# CALCULATE EMBEDDINGS FOR FEW COLUMNS WITH WEIGHTEM SUM COMBINATION
@pandas_udf("array<float>")
def get_weighted_embeddings_udf(title: pd.Series, genre: pd.Series, description: pd.Series) -> pd.Series:
    title_embeddings = model.encode(title.fillna("").tolist(), batch_size=64)
    genre_embeddings = model.encode(genre.fillna("").tolist(), batch_size=64)
    description_embeddings = model.encode(description.fillna("").tolist(), batch_size=64)
    
    # Define weights
    w_title = 0.4
    w_genre = 0.3
    w_description = 0.3

    # Weighted sum
    combined = [
        (w_title * t + w_genre * g + w_description * e).tolist()
        for t, g, e in zip(title_embeddings, genre_embeddings, description_embeddings)
    ]
    return pd.Series(combined)

/home/tim/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tim/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from pyspark.sql.functions import collect_set, concat_ws, first

BOOK_ID = '10000063'

# Filter all rows for the given book_id
book = books_df.filter(books_df['book_id'] == BOOK_ID)

merged_book = book.groupBy("book_id") \
    .agg(
        concat_ws(" ", collect_set("genre")).alias("genre"),
        first("title").alias("title"),
        first("description").alias("description"),
    )

merged_book.show(truncate=False)

+--------+------------------------------+------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# SEARCH WITH FAISS
from pyspark.sql.functions import col

book_with_vector = book.withColumn("vector", get_weighted_embeddings_udf("title", "genre", "description"))
query_vector = book_with_vector.select("vector").collect()[0]["vector"]
normalized_query = normalize(np.array(query_vector, dtype=np.float32).reshape(1, -1))

k = 5 
distances, indices = index.search(normalized_query.astype(np.float32), k)

print("Top 5 nearest neighbors ids:", indices)

def select_entries_by_row_id(row_ids, df):
    """
    Select rows from the dataframe based on the provided list of row_ids.

    Parameters:
    - row_ids: A list of row_ids to filter by.
    - df: DataFrame to select from.

    Returns:
    - A DataFrame containing the rows that match the row_ids.
    """
    row_id_set = set(row_ids)
    selected_entries = df.filter(col('faiss_id').isin(row_id_set))
    
    return selected_entries

[indices] = indices
selected_entries_df = select_entries_by_row_id(indices, songs_embedded)

selected_entries_df.show(k)


Top 5 nearest neighbors ids: [[ 15189 115843 207662 236395 190824]]
+-----------------+-----------------+-------+-------------------+--------------------+-----+--------+------+------------+------------+-----------+--------+------------+----------------+--------------------+--------+
|           artist|             song|emotion|           variance|               Genre|Tempo|Loudness|Energy|Danceability|Positiveness|Speechiness|Liveness|Acousticness|Instrumentalness|     embedded_vector|faiss_id|
+-----------------+-----------------+-------+-------------------+--------------------+-----+--------+------+------------+------------+-----------+--------+------------+----------------+--------------------+--------+
|Avenged Sevenfold|          Fiction|   fear| 0.8335142456515902|               metal|  120|   -7.45|    63|          60|          24|          3|      11|           6|               0|[-0.013874476, 0....|   15189|
|             Kygo|          Fiction|sadness| 0.8335142456515902|   